<a href="https://colab.research.google.com/github/ddPn08/kohya-sd-scripts-webui/blob/main/kohya-sd-scripts-webui-colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [kohya sd-scripts webui](https://github.com/ddPn08/kohya-sd-scripts-webui)

This notebook is for running [sd-scripts](https://github.com/kohya-ss/sd-scripts) by [Kohya](https://github.com/kohya-ss).

このノートブックは[Kohya](https://github.com/kohya-ss)さんによる[sd-scripts](https://github.com/kohya-ss/sd-scripts)を実行するためのものです。

# Repository
[kohya_ss/sd-scripts](https://github.com/kohya-ss/sd-scripts)

In [ ]:
! nvidia-smi
! nvcc -V
! free -h

In [ ]:
# @markdown # Mount Google Drive
mount_gdrive = True  # @param {type:"boolean"}
gdrive_preset_path = "/content/drive/MyDrive/AI/kohya-sd-scripts-webui/presets.json"  # @param {type:"string"}

if mount_gdrive:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)

In [ ]:
# @markdown # Initialize environment

! git clone https://github.com/ddPn08/kohya-sd-scripts-webui.git

import os

if not os.path.exists(gdrive_preset_path):
    os.makedirs(os.path.dirname(gdrive_preset_path), exist_ok=True)
    with open(gdrive_preset_path, "w") as f:
        f.write("{}")

! rm -f ./kohya-sd-scripts-webui/presets.json
! ln -s {gdrive_preset_path} ./kohya-sd-scripts-webui/presets.json

conda_dir = "/opt/conda"  # @param{type:"string"}
conda_bin = os.path.join(conda_dir, "bin", "conda")

if not os.path.exists(conda_bin):
    ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh
    ! chmod +x Miniconda3-latest-Linux-x86_64.sh
    ! bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p {conda_dir}
    ! rm Miniconda3-latest-Linux-x86_64.sh

def run_script(s):
    ! {s}

def make_args(d):
    arguments = ""
    for k, v in d.items():
        if type(v) == bool:
            arguments += f"--{k} " if v else ""
        elif type(v) == str and v:
            arguments += f"--{k} \"{v}\" "
        elif v:
            arguments += f"--{k}={v} "
    return arguments

In [ ]:
# @markdown # Run

# @markdown <br>

# @markdown ## Optional | Ngrok Tunnel
# @markdown Get token from [here](https://dashboard.ngrok.com/get-started/your-authtoken)

ngrok_token = ""  # @param {type:"string"}
ngrok_region = "us"  # @param ["us", "eu", "au", "ap", "sa", "jp", "in"]

arguments = {
    "ngrok": ngrok_token,
    "ngrok-region": ngrok_region,
    "share": ngrok_token is None,
    "xformers": True,
    "enable-console-log": True
}

run_script(f"""
eval "$({conda_bin} shell.bash hook)"
cd kohya-sd-scripts-webui
python launch.py {make_args(arguments)}
""")